In [2]:
from os import getenv
from datetime import datetime, timezone
import json
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_community.chat_models.cloudflare_workersai import ChatCloudflareWorkersAI

load_dotenv()

True

In [3]:
llm = ChatCloudflareWorkersAI(
    account_id=getenv("CF_ACCOUNT_ID"),
    api_token=getenv("CF_API_TOKEN"),
    model="@hf/nousresearch/hermes-2-pro-mistral-7b",
)

In [4]:
@tool
def date_time() -> str:
    """
    Function Name: date_time
    Parameters: None
    Description: Returns the current UTC date and time in ISO format.
    """
    return datetime.now(timezone.utc).isoformat()

tools = {
    "date_time": date_time
}

llm = llm.bind_tools(tools.values())

In [5]:


query = "Using the date_time tool, what is the current date?"
messages = [HumanMessage(query)]

ai_msg = llm.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = tools[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

response = llm.invoke(messages).content
json.loads(response.replace("'", '"').lower())["result"]["response"]

2025-03-29 22:15:41 - INFO - Sending prompt to Cloudflare Workers AI: {'prompt': 'role: user, content: Using the date_time tool, what is the current date?', 'tools': [{'type': 'function', 'function': {'name': 'date_time', 'description': 'Function Name: date_time\nParameters: None\nDescription: Returns the current UTC date and time in ISO format.', 'parameters': {'properties': {}, 'type': 'object'}}}]}
2025-03-29 22:15:43 - INFO - Sending prompt to Cloudflare Workers AI: {'prompt': "role: user, content: Using the date_time tool, what is the current date?\nrole: assistant, content: {'result': {'response': None, 'tool_calls': [{'arguments': {}, 'name': 'date_time'}], 'usage': {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}}, 'success': True, 'errors': [], 'messages': []}, tools: [{'name': 'date_time', 'arguments': {}}]\nrole: tool, content: 2025-03-30T02:15:43.630162+00:00, tool_call_id: 6acde0d7-a2b8-4b6a-b9a8-140bfeb3db01", 'tools': [{'type': 'function', 'function': {'na

'the current date, using the date_time tool, is march 30, 2025.'